In [43]:
from sklearn.model_selection import fit_grid_point
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

import pandas as pd

In [48]:
X, y = make_regression(n_samples=100)

In [53]:
rf = RandomForestRegressor(oob_score=True)

In [54]:
def oob_scorer(estimator, X, y):
    return estimator.oob_score_


fit_grid_point(X=X, 
               y=y, 
               estimator=rf, 
               parameters={'n_estimators': 100, 'max_features': 0.33}, 
               train=range(100), 
               test=None, 
               scorer=oob_scorer, 
               verbose=0)

(0.3275469307493386, {'max_features': 0.33, 'n_estimators': 100}, 1)

In [163]:
from sklearn.model_selection import RandomizedSearchCV, BaseCrossValidator
import numpy as np

params = {
    'n_estimators': [100, 200, 300, 400],
    'max_features': ['log2', 'sqrt', 0.333]
}
    
rs = RandomizedSearchCV(estimator=rf, 
                        cv=OneFold(), 
                        scoring=oob_scorer,
                        return_train_score=False, 
                        param_distributions=params, 
                        n_iter=1)
rs.fit(X, y)

pd.DataFrame(rs.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,param_max_features,param_n_estimators,params,rank_test_score,split0_test_score,std_fit_time,std_score_time,std_test_score
0,0.136392,0.00001,0.297329,0.333,100,"{'n_estimators': 100, 'max_features': 0.333}",1,0.297329,0.0,0.0,0.0


In [169]:
class _OneFold:
    def split(self, X=None, y=None, groups=None):
        return [(np.array(np.arange(_num_samples(X))), 
                 np.array(np.arange(_num_samples(X))))]

    def get_n_splits(self, X=None, y=None, groups=None):
        return 1

class RandomizedSearchOOB(RandomizedSearchCV):
    
    def __init__(self, 
                 estimator, 
                 param_distributions, 
                 n_iter=10, 
                 scoring=None, 
                 fit_params=None, 
                 n_jobs=1, 
                 iid=True, 
                 refit=True, 
                 verbose=0, 
                 pre_dispatch='2*n_jobs', 
                 random_state=None, 
                 error_score='raise'):
        
        cv = _OneFold()
        return_train_score=False
        
        super().__init__(estimator, 
                         param_distributions, 
                         n_iter, 
                         scoring, 
                         fit_params, 
                         n_jobs, 
                         iid, 
                         refit, 
                         cv, 
                         verbose, 
                         pre_dispatch, 
                         random_state, 
                         error_score, 
                         return_train_score)

        
rs = RandomizedSearchOOB(estimator=rf, 
                         scoring=oob_scorer,
                         param_distributions=params, 
                         n_iter=10)
rs.fit(X, y)

pd.DataFrame(rs.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,param_max_features,param_n_estimators,params,rank_test_score,split0_test_score,std_fit_time,std_score_time,std_test_score
0,0.329675,0.000012,0.150713,log2,400,"{'n_estimators': 400, 'max_features': 'log2'}",7,0.150713,0.0,0.0,0.0
1,0.250630,0.000011,0.314231,0.333,200,"{'n_estimators': 200, 'max_features': 0.333}",1,0.314231,0.0,0.0,0.0
2,0.279051,0.000013,0.211134,sqrt,300,"{'n_estimators': 300, 'max_features': 'sqrt'}",4,0.211134,0.0,0.0,0.0
3,0.099758,0.000035,0.167679,sqrt,100,"{'n_estimators': 100, 'max_features': 'sqrt'}",6,0.167679,0.0,0.0,0.0
4,0.169982,0.000012,0.135616,log2,200,"{'n_estimators': 200, 'max_features': 'log2'}",8,0.135616,0.0,0.0,0.0
5,0.257405,0.000013,0.117579,log2,300,"{'n_estimators': 300, 'max_features': 'log2'}",9,0.117579,0.0,0.0,0.0
6,0.187065,0.000012,0.168666,sqrt,200,"{'n_estimators': 200, 'max_features': 'sqrt'}",5,0.168666,0.0,0.0,0.0
7,0.387007,0.000011,0.300819,0.333,300,"{'n_estimators': 300, 'max_features': 0.333}",2,0.300819,0.0,0.0,0.0
8,0.080686,0.000011,0.091011,log2,100,"{'n_estimators': 100, 'max_features': 'log2'}",10,0.091011,0.0,0.0,0.0
9,0.129795,0.000010,0.281207,0.333,100,"{'n_estimators': 100, 'max_features': 0.333}",3,0.281207,0.0,0.0,0.0
